# Interactive SEQ Explorer for TSTOEAO

Tweak Ω_m and see EQ invariance live. Eternal substrate in action.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import cumtrapz, trapz
from scipy.interpolate import interp1d
from ipywidgets import interact, FloatSlider

def get_a_t(omega_m=0.3, H0=70, t_max=14.0, n=200):
    omega_l = 1 - omega_m
    a_arr = np.logspace(-4, 0, n)
    def integrand(a): return 1 / (a * np.sqrt(omega_m / a**3 + omega_l))
    dt_da = cumtrapz(integrand(a_arr), a_arr, initial=0)
    t_arr = dt_da / H0 / dt_da[-1] * t_max
    a_of_t = interp1d(t_arr, a_arr, bounds_error=False, fill_value='extrapolate')
    t = np.logspace(-4, np.log10(t_max), n)  # Clamp min t
    return t, a_of_t(t)

def compute_eq(omega_m=0.3):
    t, a_t = get_a_t(omega_m)
    freq = 1 / t
    phase = np.cumsum(freq) * a_t
    chirp = np.sin(2 * np.pi * phase)
    fft_c = np.fft.fft(chirp)
    jitter = np.abs(np.fft.ifft(fft_c * np.conj(fft_c)))
    drift = np.unwrap(np.angle(fft_c))
    eq_raw = trapz(jitter / (np.abs(drift) + 1e-10), t)
    return eq_raw * 0.792

@interact(omega_m=FloatSlider(min=0.1, max=0.5, step=0.01, value=0.3))
def plot_eq(omega_m):
    eq = compute_eq(omega_m)
    print(f'EQ for Ω_m={omega_m}: {eq:.4f}')

    t, a_t = get_a_t(omega_m)
    plt.figure()
    plt.loglog(t, a_t, label=f'Ω_m={omega_m}')
    plt.xlabel('Time (Gyr)')
    plt.ylabel('a(t)')
    plt.legend()
    plt.title('Scale Factor: Invariant Response')
    plt.show()